In [1]:
import os
os.getcwd()
import sqlite3 as sq

In [2]:
conn = sq.connect("crude_oil.db")

In [3]:
# Create table called crudeOilPrice
query = """
CREATE TABLE crudeOilPrice(
    year VARCHAR(4),
    month SMALLINT,
    crude_oil_price DECIMAL(5,2),
    production DECIMAL(3,2),
    crude_oil_export DECIMAL(3,2)
    )"""

with conn:
    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS crudeOilPrice")
    cur.execute(query)

In [4]:
# checking our sqlite master for confirmation
query = "SELECT name FROM sqlite_master WHERE type='table'"
cur.execute(query)
print(cur.fetchall())

[('crudeOilPrice',)]


In [5]:
# previewing the table content
query = "SELECT * FROM crudeOilPrice"
cur.execute(query)
print(cur.fetchall())

[]


In [6]:
# coverting sql table to dataframe
import pandas as pd

sqlDF = pd.read_sql_query(query, conn)
sqlDF

,year,month,crude_oil_price,production,crude_oil_export


Let fill in values from the csv


In [7]:
# Loading CSV file
csv = pd.read_csv('CrudeOilProdPrice25102022.csv', index_col=False)
csv.head()

,Year,Month,Crude Oil Price,Production,Crude Oil Export
0,2006,1,63.85,2.59,2.14
1,2006,2,61.33,2.47,2.02
2,2006,3,65.00,2.25,1.80
3,2006,4,72.09,2.32,1.87
4,2006,5,71.18,2.28,1.83


In [8]:
# creating a tuple of values in each row
csvTuple = tuple(csv.itertuples(index=False, name=None))
csvTuple[0]

(2006, 1, 63.85, 2.59, 2.14)

In [9]:
# inserting into table on the database
insertQuery = "INSERT INTO crudeOilPrice values(?,?,?,?,?)"
cur.executemany(insertQuery, csvTuple)

In [10]:
# Previewing table
query = "SELECT * FROM crudeOilPrice"

# Reading table as a pandas dataframe
sqlDF = pd.read_sql_query(query, conn)
sqlDF

,year,month,crude_oil_price,production,crude_oil_export
0,2006,1,63.85,2.59,2.14
1,2006,2,61.33,2.47,2.02
2,2006,3,65.00,2.25,1.80
3,2006,4,72.09,2.32,1.87
4,2006,5,71.18,2.28,1.83
...,...,...,...,...,...
196,2022,5,116.72,1.02,0.57
197,2022,6,130.10,1.16,0.71
198,2022,7,120.54,1.08,0.63
199,2022,8,106.34,0.97,0.52


## Data Cleaning

In [11]:

query = """
UPDATE
    crudeOilPrice
SET 
    month = 
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(month, 12, 'December'), 
        2, 'February'), 
        3, 'March'),
        4, 'April'),
        5, 'May'), 
        6, 'June'),
        7, 'July'),
        8, 'August'),
        9, 'September'),
        10, 'October'),
        11, 'November'),
        1, 'January')
"""

cur.execute(query)

In [12]:
query = "SELECT * FROM crudeOilPrice"
pd.read_sql_query(query, conn)

,year,month,crude_oil_price,production,crude_oil_export
0,2006,January,63.85,2.59,2.14
1,2006,February,61.33,2.47,2.02
2,2006,March,65.00,2.25,1.80
3,2006,April,72.09,2.32,1.87
4,2006,May,71.18,2.28,1.83
...,...,...,...,...,...
196,2022,May,116.72,1.02,0.57
197,2022,June,130.10,1.16,0.71
198,2022,July,120.54,1.08,0.63
199,2022,August,106.34,0.97,0.52


In [13]:
query = """
SELECT DISTINCT month FROM crudeOilPrice
"""
sqlDF = pd.read_sql_query(query, conn)
sqlDF

,month
0,January
1,February
2,March
3,April
4,May
5,June
6,July
7,August
8,September
9,October


In [30]:
query = """
SELECT DISTINCT year FROM crudeOilPrice
"""
sqlDF = pd.read_sql_query(query, conn)
sqlDF

,year
0,2006
1,2007
2,2008
3,2009
4,2010
5,2011
6,2012
7,2013
8,2014
9,2015


In [15]:
query = """
SELECT AVG(crude_oil_price) AS average_oil_price
FROM crudeOilPrice
"""
sqlDF = pd.read_sql_query(query, conn)
sqlDF

,average_oil_price
0,78.167463


In [16]:
query = """
SELECT AVG(production) AS average_production
FROM crudeOilPrice
"""
sqlDF = pd.read_sql_query(query, conn)
sqlDF

,average_production
0,2.031741


In [17]:
query = """
SELECT SUM(crude_oil_export) AS total_crude_oil_exported
FROM crudeOilPrice
"""
sqlDF = pd.read_sql_query(query, conn)
sqlDF

,total_crude_oil_exported
0,317.93


In [29]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September']

query = """
SELECT month, SUM(production) AS total_production
FROM crudeOilPrice
WHERE year = '2022'
GROUP BY month
"""
sqlDF = pd.read_sql_query(query, conn, index_col='month').reindex(month_order)
print(sqlDF)

import plotly.express as px
px.line(sqlDF,
        x=sqlDF.index, width=700,
        y= 'total_production',
        title="2022 Monthly Crude Oil Production") 

           total_production
month                      
January                1.40
February               1.26
March                  1.24
April                  1.22
May                    1.02
June                   1.16
July                   1.08
August                 0.97
September              0.94


In [19]:
query = """
SELECT month, SUM(production) AS total_production, year
FROM crudeOilPrice
WHERE year = '2022'
GROUP BY month
"""
sqlDF = pd.read_sql_query(query, conn)
print(sqlDF)

       month  total_production  year
0      April              1.22  2022
1     August              0.97  2022
2   February              1.26  2022
3    January              1.40  2022
4       July              1.08  2022
5       June              1.16  2022
6      March              1.24  2022
7        May              1.02  2022
8  September              0.94  2022


In [20]:
query = """
SELECT month, SUM(production) AS total_production, year
FROM crudeOilPrice
WHERE year = '2021'
GROUP BY month
"""
sqlDF = pd.read_sql_query(query, conn)
print(sqlDF)

        month  total_production  year
0       April              1.37  2021
1      August              1.24  2021
2    December              1.20  2021
3    February              1.42  2021
4     January              1.36  2021
5        July              1.32  2021
6        June              1.31  2021
7       March              1.43  2021
8         May              1.34  2021
9    November              1.23  2021
10    October              1.23  2021
11  September              1.25  2021


In [21]:
query = """
SELECT month, SUM(production) AS total_production, year
FROM crudeOilPrice
GROUP BY month
"""
sqlDF = pd.read_sql_query(query, conn)
print(sqlDF)

        month  total_production  year
0       April             34.24  2006
1      August             34.27  2006
2    December             32.15  2006
3    February             35.62  2006
4     January             35.65  2006
5        July             34.16  2006
6        June             33.51  2006
7       March             34.26  2006
8         May             33.64  2006
9    November             32.57  2006
10    October             33.95  2006
11  September             34.36  2006


In [22]:
import plotly.graph_objs as go

In [23]:
# Filtering monthly production
query = """
SELECT c1.month, SUM(c1.production) AS total_prod_2021, sub.total_prod_2022
FROM crudeOilPrice AS c1,
                        (SELECT month, SUM(production) AS total_prod_2022
                        FROM crudeOilPrice
                        WHERE year = '2022'
                        GROUP BY month) AS sub
WHERE year = '2021' AND (c1.month = sub.month) 
GROUP BY c1.month
"""
sqlDF = pd.read_sql_query(query, conn, index_col='month').reindex(month_order)
print(sqlDF)

           total_prod_2021  total_prod_2022
month                                      
January               1.36             1.40
February              1.42             1.26
March                 1.43             1.24
April                 1.37             1.22
May                   1.34             1.02
June                  1.31             1.16
July                  1.32             1.08
August                1.24             0.97
September             1.25             0.94


In [24]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sqlDF.index, y=sqlDF.total_prod_2021,
                         name='2021 Production', line=dict(color='firebrick', 
                                                           width=4, dash='dot')))
fig.add_trace(go.Scatter(x=sqlDF.index, y=sqlDF.total_prod_2022,
                         name='2022 Production', line=dict(color='royalblue', 
                                                           width=4, dash='dot')))
fig.update_layout(title='Crude Oil Production (mbd)',
                  yaxis_title = 'Millions Barrels Per Day (mbd)',
                  )
fig.show()